# Geolocalización

En esta notebook haremos un intento de geolocalización con los textos de los usuarios...

In [1]:
from pymongo import MongoClient

client = MongoClient('localhost')

db = client['contrastes']

In [2]:
db.tweets.find_one()

{'_id': ObjectId('5ba53c9827a5141aaa383eb9'),
 'created_at': 'Tue Nov 05 14:48:51 +0000 2013',
 'id': 397737276736040960,
 'place': None,
 'provincia': 'larioja',
 'text': 'Estoy tan asustada :(',
 'tokens': ['estoy', 'tan', 'asustada'],
 'user_id': 301800629}

In [3]:
user_ids = list(db.users.distinct('id'))
print("Tenemos {} usuarios".format(len(user_ids)))

Tenemos 56308 usuarios


Hagamos lo siguiente:

- Entrenemos con unigramas una regresión logística para 
- Luego probemos con los regionalismos

Primero, partamos en train, test

In [4]:
import sklearn
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

np.random.seed(2019)

sample_user_ids = random.sample(user_ids, 5000)

# Nos quedamos sólo con los campos que nos interesan
users = list(db.users.find({"id": {"$in": sample_user_ids}}, {"id": 1, "_id": 0, "text": 1, "provincia": 1}))



train_users, test_users = train_test_split(users)

df_train = pd.DataFrame(train_users)
df_train.set_index("id", inplace=True)


df_test = pd.DataFrame(test_users)
df_test.set_index("id", inplace=True)

df_train.groupby("provincia").count()


,text
provincia,
buenosaires,165
catamarca,165
chaco,162
chubut,145
cordoba,167
corrientes,182
entrerios,174
formosa,161
jujuy,155


## Palabras precalculadas

Carguemos antes las palabras que sabemos que ocurren una cantidad razonable de veces

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
vectorizer = CountVectorizer(
    tokenizer=tokenizer.tokenize, stop_words=stopwords.words('spanish'),
    min_df=0.0001, max_df=0.15, ngram_range=(1, 2),
)

vectorizer.fit(df_train["text"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.15, max_features=None, min_df=0.0001,
        ngram_range=(1, 2), preprocessor=None,
        stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde'...tuviésemos', 'tuvieseis', 'tuviesen', 'teniendo', 'tenido', 'tenida', 'tenidos', 'tenidas', 'tened'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fe7d652b160>>,
        vocabulary=None)

In [6]:
print("Vocabulario del vectorizador: {} palabras".format(len(vectorizer.vocabulary_)))

Vocabulario del vectorizador: 12957624 palabras


In [7]:
X_train = vectorizer.transform(df_train["text"])
X_test = vectorizer.transform(df_test["text"])

In [8]:
from sklearn.preprocessing import LabelEncoder

province_encoder = LabelEncoder()

province_encoder.fit(df_train["provincia"].values)

LabelEncoder()

In [9]:
y_train = province_encoder.transform(df_train["provincia"].values)
y_test = province_encoder.transform(df_test["provincia"].values)

La reg. logística será un softmax, así que elijo `multi_class='multinomial'`

In [10]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='saga')

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

32% de accuracy

## Usando sólo "regionalismos" o LIW (Location Indicative Words)

Usemos ahora nuestros "features". Es decir, probemos con porcentajes de las palabras encontradas

In [ ]:
df_words = pd.read_csv("../output/listados/listado_completo.csv")
df_words.set_index("palabra", inplace=True)
df_words.sort_values("rank_personas", ascending=True, inplace=True)

df_words.iloc[:10]

Veamos qué performance tiene usando 1000, 2000, 3000, y así...

In [ ]:
from sklearn.linear_model import LogisticRegression

clfs = {}
scores = {}

for num_words in range(250, 30000, 250):    
    liw_vectorizer = CountVectorizer(
        tokenizer=tokenizer.tokenize,
        vocabulary=df_words.index[:num_words])

    X_train = liw_vectorizer.transform(df_train["text"])
    X_test = liw_vectorizer.transform(df_test["text"])

    clf = LogisticRegression(multi_class='multinomial', solver='saga')
    clf.fit(X_train, y_train)
    
    scores[num_words] = clf.score(X_test, y_test)
    clfs[num_words] = clf
    

Esto me da miedo...

In [ ]:

scores